# ETL Project

## Extract

From Kaggle.com, we found two sources of data in .csv form that had similar subjects; famous movie catchphrases and movies available on netflix. The common column in each file is 'title' (the title of the movie). We thought it'd be fun to be able to find which of these famous movie catchphrases were available on Netflix.

We read these .csv files with the Python Pandas library and assigned them to their own respective dataframes.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import datetime
from config import (user, password, host, port, database)

In [ ]:
# read catchphrase.csv file
csv_file = "catchphrase.csv"

# store .csv into dataframe
catchphrases_df = pd.read_csv(csv_file)
catchphrases_df.head()

In [ ]:
# read catchphrase.csv file
csv_file = "netflix_titles.csv"

# store .csv into dataframe
netflix_df = pd.read_csv(csv_file)
netflix_df

## Transform

To transform the data, we made sure the column names across both dataframes has consistent lowercase snake-case formatting.

Because the data in the 'title' column of the original .csv movie catchrases file were in ALL CAPS, we decided to make the 'title' column in the netflix dataframe to be same. To do so, we used the str.upper() function.

For good measure, we dropped duplicates in the netflix dataframe because there were so many records. We dropped a number of columns in each dataframe we decided were not relevant for particularly useful for this query.

Finally, we set the 'title' column in each dataframe to the index. This way, we could rely on serial index columns in the schema we created on the SQL end.

In [ ]:
# rename columns of catchphrase_df
catchphrases_df = catchphrases_df.rename(columns={'Catchphrase': 'catchphrase',
                                'Movie Name': 'title',
                                'Context': 'context'})
catchphrases_df

In [ ]:
# drop duplicates of netflix_df, jut in case.
netflix_df = netflix_df.drop_duplicates()
netflix_df.head()

In [ ]:
# change netflix titles to ALL CAPS to be consistent with catchphrase_df
netflix_df['title'] = netflix_df['title'].str.upper()
netflix_df.head()

In [ ]:
# drop unwanted columns in nextflix_df
netflix_df = netflix_df[['title', 'date_added', 'release_year', 'rating', 'duration']]

# set 'title' as index
netflix_df = netflix_df.set_index('title')
netflix_df.head()

In [ ]:
# drop unwanted columns from catchphrase_df
catchphrases_df = catchphrases_df[['title', 'catchphrase']]

# set 'title' as index
catchphrases_df = catchphrases_df.set_index('title')
catchphrases_df.head()

## Load
We took the dataframes we created and loaded them into the respective tables we created in PostgreSQL.

In [ ]:
# create engine to postgreSQL
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
conn = engine.connect()

In [ ]:
# look at tables in database
engine.table_names()

In [ ]:
# load catchphrases_df into catchphrases_db
catchphrases_df.to_sql(name='catchphrases', con=engine, if_exists='append', index=True)

In [ ]:
# load netflix_df into netflix_db
netflix_df.to_sql(name='netflix', con=engine, if_exists='append', index=True)